# Сравнение open-source солверов и их применение в ритейле

Привет, Habr! На связи отдел аналитики данных X5 Tech.

Сегодня мы продолжим говорить про оптимизаторы.
Для каких задач и как они применяются мы поговорили в [предыдущей статье](http://).

Сегодня же мы сделаем небольшой обзор существующих опен-сурс решений, затронем их различия, типы задач, а также сделаем небольшое сравнение
решения модельной задачи.


## Обзор пакетов python


Рассмотрим open-source пакеты и солверы для решения задач условной оптимизации в зависимости от ее типа, которые часто можно встретить в практических задачах.

### Открытые библиотеки, предоставляющие интерфейс для решения оптимизационных задач


[__Scipy__](https://scipy.org/) - библиотека, которая содержит большой набор функций для научных вычислений,
в том числе имеет инструменты для решения оптимизационных задач, находящиеся в модуле scipy.optimize.
В модуле находятся методы для решения задач линейного программирования (__LP__), нелинейного программирования (__NLP__).

Солверы, которые поддерживают условную и безусловную задачи оптимизации:

  1)  ___cobyla___
  2)  ___slsqp___
  3)  ___trust-constr___

Целочисленной оптимизации не поддерживается.
Краткое описание методов и ссылки на статьи можно найти [тут](https://habr.com/ru/company/ods/blog/448054/).

[__Pyomo__](http://www.pyomo.org/) - пакет, который содержит ряд инструментов для формулирования, решения и анализа оптимизационных моделей.
Главная особенность — это удобный интерфейс для структурированного формулирования оптимизационной задачи и поддержка большого количества солверов, в том числе коммерческих.
Pyomo преобразует сформулированную модель в формат, понятный для запускаемого солвера.

Pyomo входит в проект [COIN-OR](https://www.coin-or.org/), содержащий ряд солверов, среди которых выделим два:

  1) [___Ipopt___](https://github.com/coin-or/Ipopt) - находит локальные оптимумы в задаче NLP с помощью прямо-двойственного метода внутренней точки, подробнее в оригинальной [статье](http://www.optimization-online.org/DB_HTML/2004/03/836.html).
  2) [___Cbc___](https://github.com/coin-or/Cbc) - решает задачи MILP, на базе алгоритма, сочетающем в себе метод ветвей и границ и секущих плоскостей [wiki](https://en.wikipedia.org/wiki/Branch_and_cut).

Также для LP имеется поддержка пакета [glpk](https://en.wikipedia.org/wiki/GNU_Linear_Programming_Kit).
И напоследок отметим еще один солвер [bonmin](https://github.com/coin-or/Bonmin), который построен поверх __Cbc__ и __Ipopt__ - сочетание двух солверов,
позволяющее браться за задачи __MINLP__.

[__Cvxpy__](https://www.cvxpy.org/index.html) - данный пакет специально заточен для решения задач выпуклой оптимизации (convex optimization).
После того как задача сформулирована, перед решением проверяется выпуклость и аффинность целевой функции и ограничений с помощью правил [DCP](https://www.cvxpy.org/tutorial/dcp/index.html)
(disciplined convex programming). 
После проверки задача преобразуется в стандартную форму и передается квадратичному или коническому солверу.
Полный список солверов и решаемых задач можно найти [здесь](https://www.cvxpy.org/tutorial/advanced/index.html).

Составим таблицу для перечисленных пакетов и солверов, какие типы задач они решают:


|  Пакеты в python  |     Солвер(метод)    | NLP | LP | MILP | MINLP |
|-------------------|----------------------|-----|----|------|-------|
| scipy             | cobyla               | y   | n  | n    | n     |
| scipy             | slsqp                | y   | n  | n    | n     |
| scipy             | trust-constr         | y   | n  | n    | n     |
| pyomo             | ipopt                | y   | y  | n    | n     |
| pyomo, cvxpy      | glpk                 | n   | y  | y    | n     |
| pyomo, cvxpy      | cbc                  | n   | y  | y    | n     |
| cvxpy             | ecos                 | y   | y  | y    | n     |
| pyomo             | bonmin               | y   | y  | y    | y     |



## Описание подхода к решению оптимизационной задачи

Решение задач будем производить с помощью библиотек и солверов, указанных выше.
Slsqp, cobyla, trust-constr из scipy, ipopt из pyomo - будут использованы для решения задачи с непрерывными переменными (2.\*).

Cobyla не поддерживает ограничения типа равенства, коими здесь являются (2.4.1), но это не помешает нам, так как это ограничение просто равенство переменных,
а значит внутри группы можно задать всего одну переменную — она будет общей для линейки.
То же самое можно сделать и для всех остальных NLP солверов из scipy, а вот в pyomo можно явно задать равенство переменных, так в процессе подготовки он сам сделает замену на одну переменную.

Что касается cvxpy, то несложно увидеть, что исходный функционал не является выпуклым.
Более того, в общем случае, исходя из логических соображений можно прикинуть, что при увеличении цены спрос и вместе с ним выручка будут асимптотически падать до нуля,
а при уменьшении цены ближе к нулю выручка будет стремиться к какому-то большому числу.
То есть функция, описывающая выручку от цены, будет как минимум содержать выпуклость вниз (вогнутость), что при условии максимизации функции не удовлетворяет условиям выпуклого программирования.

Задача (3.\*) - будет решаться с помощью солверов glpk, cbc из пакетов pyomo, cvxpy и ecos из cvxpy.

## Пайплайн

Пайплайн решения задачи можно разбить на три этапа:
* препроцессинг, в который входит подготовка данных, формирование ценовых правил.
* расчёт — шаг с решением оптимизационной задачи.
* постпроцессинг — анализ полученного решения в случае успеха.

Третий этап необходим, т.к. может сложится, что задачу на шаге 2 не удалось решить.
Например, из-за пустого допустимого множества, т.е. множества, в котором переменные удовлетворяют всем ограничениям.
В таких случаях можно просто подсветить, что задача не имеет решения или разработать схему исключения/ослабления ограничений и вернуться к шагу 2 с обновленными условиями.
Обработка таких случаев это тоже отдельная и не всегда тривиальная задача, в этой статье касаться не будем.

### Препроцессинг

Начнем с шага 1.
Для модельной задачи ценообразования был реализован генератор данных, который по заданным параметрам формирует все необходимые данные для двух постановок задач, код генератора описан [здесь](https://github.com/mbudylin/OptimizersArticle/blob/main/data_generator/data_generator.py).
На входе — количество линеек, ценовые диапазоны, заданные словарем, размер сетки для задачи (3.\*) и seed для фиксации генератора случайных чисел.
На выходе получаем данные для задачи NLP, MILP и словарь, содержащий список товаров для каждой линейки.
При генерации данных опираемся на количество линеек, так как выше было показано, что поиск цен происходит внутри линейки, т.е. необходима одна переменная(или вектор в LP) на одну линейку.
Размер сетки для поиска в MILP постановке возьмем 21 = текущая цена и по 10 цен в каждую сторону в рамках диапазона +- 10% с дальнейшей отсечкой по границам (2.2) с дальнейшим округлением.
В результате округления можем слегка выйти за основной диапазон — будем считать это допустимым.
Для удобства выполнения матричных операций, в случае, если финальный размер сетки для товара меньше заданной, дополняем сетку нулями справа.

```python
# пример кода для запуска генератора данных
from data_generator.data_generator import generate_data

N = 5
bounds_params = {
    'main_bounds': {
        'lower': 0.9, 'upper': 1.1
    },
    'market_bounds': {
        'lower': 0.85, 'upper': 1.15
    }
}
grid_size = 21
seed = 10
data = generate_data(N, bounds_params, grid_size, seed)
```

<details>
<summary> |> Пример данных для NLP постановки </summary>
<div><pre><code class="python">
# plu_line - код продуктовой линейки
# plu - код товара
# plu_idx - индекс товара
# P - текущая цена
# Q - текущие продажи в штуках
# E - эластичность
# PC - цена конкурента/рыночная цена
# C - себестоимость товара
# x_lower - нижняя граница x для диапазона поиска цены
# x_upper - верхняя граница x для диапазона поиска цены
# x_init - начальное значение x для старта оптимизатора
# fixed - метка, необходимо ли фиксировать x_init цену на товар, по умолчанию всегда 0
data['data_nlp']

<img src="images/data_nlp_sample.png" width="550" align="center"/>
</code></pre><p></p></div>
</details>



<details>
<summary> |> Пример данных для MILP постановки </summary>
<div><pre><code class="python">
Пример данных для MILP постановки
# plu_line - код продуктовой линейки
# Ps - сетка цен для поиска
# Qs - сетка продаж для кажой цены из Ps
# xs - сетка индексов
# grid_size - размер сетки
# P_idx - индекс текущей цены в сетке. Если значение -1, то текщая цена не попала в сетку
data['nlp']

<img src="./images/data_milp_sample.png" width="1000" align="c"/>
</code></pre><p></p></div>
</details>


### Расчёт


На шаге 2 будет решаться сама оптимизационная задача.
Выглядит удобным описать оптимизационную модель в виде базового класса __OptimizationModel__,
который будет содержать методы, необходимые для постановки задачи.
Методы в классе __OptimizationModel__:
* init_objective - задание целевой функции,
* add_con_mrg - ограничений и метод,
* init_variables - задание переменных и границ.
* solve - поиск оптимального решения

Таким образом, можно организовать единый интерфейс для решения различными пакетами.
Реализация методов для каждой библиотеки будет отличаться, с чем можно ознакомиться в модуле optimizers.

Запуск оптимизационной задачи теперь можно описать в виде функции, в которую поступают данные,
модель для решения задачи, параметры ограничений — словарь и дополнительные опции для солвера [ссылка на github](https://github.com/mbudylin/OptimizersArticle/blob/main/optimizers/optimization.py).

```python
def pricing_optimization(data, opt_model, opt_params, solver, solver_option={}):
    """
    Запуск расчета оптимальных цен с помощью указанного класса оптимизатора и параметров
    :param data: входные данные для оптимизации
    :param opt_model: класс модели оптимизатора
    :param opt_params: параметры оптимизации
    :param solver: солвер для оптимизации
    :param solver_option: параметры солвера
    :return: словарь, возвращаемый моделью оптимизации
    """

    model = opt_model(data, opt_params['alpha'])

    model.init_variables()
    model.init_objective()
    # ...
    result = model.solve(solver=solver, options=solver_option)
    result['model_class'] = model
    return result
```

Через opt_params можно управлять ограничениями, включать их по необходимости, задавать границы.
Для наших задач он будет выглядеть так:

```python
opt_params = {
    'alpha': 0.0,
    'con_mrg': M_cur,
}
```


__Реализации классов для оптимизационных задач__

Описание классов здесь займет достаточно много места, детали можно найти [тут, код-github](https://github.com/mbudylin/OptimizersArticle/blob/main/optimizers/optimizers.py),
где реализовано 4 класса:
* __ScipyNlpOptimizationModel__ для NLP постановки через Scipy
* __PyomoNlpOptimizationModel__ для NLP постановки через Pyomo
* __PyomoLpOptimizationModel__ для MILP постановки через Pyomo
* __CvxpyLpOptimizationModel__ для MILP постановки через Cvxpy


Теперь можно считать, что все готово для запуска и тестирования оптимизаторов.
Рассмотрим на небольшом примере, что все отрабатывает корректно и выполняются все условия на примере двух классов оптимизационных моделей.

In [1]:
from tests.test_solvers import test_solving
test_solving()

dR = +7.9%, dM = +0.0% - Изменение выручки и маржи в NLP задаче
dR = +7.87%, dM = +0.01% - Изменение выручки и маржи в MILP задаче
dR = +7.86%, dM = +0.62% - Изменение выручки и маржи в NLP задаче после округления
Все товары в линейке имеют одинаковые цены: True


## Результаты

### Результаты для __NLP__ и __MILP__


Посмотрим за какое время и как они справятся с разным количеством товаров $n$, читай размерностью задачи.
Для этого был написан [скрипт](https://github.com/mbudylin/OptimizersArticle/blob/main/runner.py), который пробегается по значениям $n$ = \[10, 20, 50, 100, 200, 500, 1000\] и данных,
сгенерированных для набора seed'ов от 0 до 24, итого 25 тестов для каждой модели и размерности.
Будем замерять время оптимизации, обернув функцию pricing_optimization декоратором.
Во время предварительных запусков стало понятно, что некоторые оптимизаторы плохо справляются с размерностями $\gtrsim$ 100, поэтому чтобы лишний раз не греть атмосферу, было решено ограничить $n$ сверху для них.

Чтобы запустить расчёты, нужно выполнить команду:

```console
python runner.py
```

Результаты замеров представлены ниже на графиках.

<img src="./images/time_solve_compare.png" width="1000" align="c"/>

### Доля успешно решенных задач

<img src="./images/success_rate.png" width="200" align="c"/>

Жирной линией отображена медиана, а полупрозрачный коридор — это нижняя и верхняя квартили.

По задаче NLP видно, что на низкоразмерных задачах (<100), slsqp(scipy) вполне себе является лидером,
а вот при больших размерностях, явно побеждает ipopt(pyomo).

Trust-constr уже на старте всем уступает.

Для MILP задачи асимптотики у всех плюс-минус похожи, интересно что для одного и того же солвера пакеты pyomo и cvxpy дают результаты отличающиеся по времени работы.
Одна из причин этого в том, что pyomo во время объявления переменных, целевой функции и ограничений часть времени тратит н на преобразование в вид, который позволит передать задачу дальше во внешний солвер.
Это может сильно повлиять на общее время решения, особенно это заметно, когда объявляется большое число ограничений.

### Результаты для __MINLP__


Ограничение (3.5) реализовано в методе PyomoNlpOptimizationModel.add_con_chg_cnt.


<img src="./images/time_solve_minlp.png" width="900" align="c"/>


Как можно видеть, время решения задачи серьезно увеличивается, что не удивительно, саму по себе задачу NLP не всегда просто решить,
а тут еще нужно учесть дискретные правила, которые увеличивают сложность, куда гармоничнее эти правила укладываются в MILP постановку,
так как такие задачи достаточно хорошо научились решать, что нельзя сказать про MINLP.


## Заключение

В данной статье мы сравнили возможности и производительность нескольких открытых солверов на модельной задаче.
Мы надеемся, что данная статья как-то облегчит жизнь тем, кто на практике сталкивается с моделированием подобных задач.
Также отметим, что рассмотрены не все существующие солверы, а те, для которых есть хорошая документация, и которые мы использовали на практике.
Будем рады услышать про ваш опыт реализации оптимизаторов в комментариях.
Над статьёй работали Михаил Будылин, Антон Денисов, Михаил Дубровский. 
